<a href="https://www.kaggle.com/code/franciscomesquita/ad-tp1-skin-lesions-images-classification?scriptVersionId=102879665" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Imports e preparação 

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import pandas as pd 
import numpy as np
import seaborn as sns
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
#Não funciona no kaggle (apenas funcionou no colab)

!pip install visualkeras
import visualkeras

In [ ]:
#Google Drive mount
# from google.colab import drive
# drive.mount('/content/kaggle/skindiseasedataset')

In [ ]:
# !unzip -q '/content/drive/MyDrive/ColabNotebooks/DatasetSkinDisease.zip'

In [ ]:
# Obter e preparar o dataset
import pathlib

#COLAB
#data_dir = pathlib.Path('/content/Dataset skin disease v2/Images')

#Kaggle
data_dir = pathlib.Path('../input/skindiseasedataset/Dataset skin disease v2/Images')

# Input size initial
# batch_size = 32
# IMG_SIZE = (180, 180)

# Input size Final
batch_size = 64
IMG_SIZE = (32, 32)

# Input size AlexNet
# batch_size = 32
# IMG_SIZE = (227, 227)

# Input size MobileNet e VGG-16
# batch_size = 32
# IMG_SIZE = (224, 224)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  shuffle=True,
  image_size=IMG_SIZE,
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  shuffle=True,
  image_size=IMG_SIZE,
  batch_size=batch_size)

class_names = train_ds.class_names

print(class_names)


In [ ]:
def class_distribution(dataset):
    class_values = []
    total_batches = dataset.__len__().numpy()
    for batch, element in enumerate(dataset.as_numpy_iterator()):
        if batch+1 == total_batches:
            for i in range(len(element[1])):
                class_values.append(class_names[int(element[1][i])])
        else:
            for i in range(batch_size):
                class_values.append(class_names[int(element[1][i])])

    class_n, frequency = np.unique(np.array(class_values), return_counts=True)
    # frequency = frequency/len(class_values)
    count = pd.DataFrame(columns=["Classes","Count"])
    count["Classes"] = class_n
    count["Count"] = frequency
    return count, class_values

dataset1 = tf.keras.preprocessing.image_dataset_from_directory(data_dir)

df_class_dist, class_df_values_val = class_distribution(dataset1)

sns.barplot(data=df_class_dist, y="Count" ,x="Classes");

In [ ]:
train_ds = train_ds.cache().prefetch(1)
val_ds = val_ds.cache().prefetch(1)

#test dataset
val_batches = val_ds.cardinality().numpy()

test_ds = val_ds.take(val_batches // 3) 
val_ds = val_ds.skip(val_batches // 3)

In [ ]:
normalization_layer = keras.layers.experimental.preprocessing.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
# Obter alguma informação sobre o dataset

#Cardinalidade 
print('Cardinalidade Treino: ', train_ds.cardinality().numpy())
print('Cardinalidade Validacão: ', val_ds.cardinality().numpy())
print('Cardinalidade Teste: ', test_ds.cardinality().numpy())

print('Classes: ', class_names)

In [ ]:
plt.figure(figsize=(15, 10))
for images, labels in train_ds.take(1):
    for i in range(10):
        ax = plt.subplot(2, 10, i + 1)
        plt.imshow(images[i].numpy().astype("float32")) #pode ser transformado num array numpy 
        plt.title(class_names[labels[i]])
        plt.axis("off")

# Tensorboard

Para visualizar o tensorboard deve-se passar por parâmetro o tensorboard_cb ao fazer fit do modelo

In [ ]:
# Utilização da Ferramenta TensorBoard
# Diretoria Raiz
root_logdir = os.path.join(os.curdir, "my_logs")

# Sub-diretoria onde ficarão os dados da execução: tem um nome criado a partir da hora de inicio
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

# Criação do callback para executar a recolha de dados para o Tensorboard durante o treino
# Consultar os parâmetros aqui: https://keras.io/api/callbacks/tensorboard/
run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir, histogram_freq=1)

# Arquitetura Inicial

In [ ]:
# Arquitetura inicial de uma CNN

inputs = keras.Input(shape=(180,180,3))
x = keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu')(inputs)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(48, 3, activation='relu')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
outputs = keras.layers.Dense(7, activation="softmax")(x)  

model = keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
model.summary()

In [ ]:
L = 'sparse_categorical_crossentropy'

model.compile(loss=L, optimizer="adam", metrics=["accuracy"])

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True)

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100,
  callbacks=[early_stopping_cb]
)

In [ ]:
model.evaluate(test_ds)

In [ ]:
visualkeras.layered_view(model, legend=True, scale_xy=1)

In [ ]:
# Visualizar a analisar os resultados
# Visualização da evolução da accurary e da loss
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

# Arquitetura Final

In [ ]:
# Data augmention
data_augmentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    keras.layers.experimental.preprocessing.RandomFlip("vertical"),
    keras.layers.experimental.preprocessing.RandomRotation(0.5),
    keras.layers.experimental.preprocessing.RandomZoom(0.4),
])

In [ ]:
# Arquitetura final de uma CNN
inputs = keras.Input(shape=(32,32,3))
x = data_augmentation(inputs)
x = keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding = 'same')(x)
x = keras.layers.Conv2D(32, 3, activation='relu', padding = 'same')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(64, 3, activation='relu', padding = 'same')(x)
x = keras.layers.Conv2D(64, 3, activation='relu', padding = 'same')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(128, 3, activation='relu', padding = 'same')(x)
x = keras.layers.Conv2D(128, 3, activation='relu', padding = 'same')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(256, 3, activation='relu', padding = 'same')(x)
x = keras.layers.Conv2D(256, 3, activation='relu', padding = 'same')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dense(48, activation='relu')(x)
outputs = keras.layers.Dense(7, activation="softmax")(x)

model1 = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model1.summary()

In [ ]:
L = 'sparse_categorical_crossentropy'

model1.compile(loss=L, optimizer="adam", metrics=["accuracy"])

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True)

history1 = model1.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100,
  callbacks=[early_stopping_cb]
)

In [ ]:
model1.evaluate(test_ds)

In [ ]:
visualkeras.layered_view(model1, legend=True)

In [ ]:
# Visualizar a analisar os resultados
# Visualização da evolução da accurary e da loss
history1_frame = pd.DataFrame(history1.history)
history1_frame.loc[:, ['loss', 'val_loss']].plot()
history1_frame.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
# Lançar extensão TensorBoard
%load_ext tensorboard

In [ ]:
# Iniciar o servidor TensorBoard 
# --port=6006
# --host=127.0.0.1
%tensorboard --logdir=./my_logs --host=127.0.0.1

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

# Testes de Arquiteturas conhecidas

Em cada arquitetura tivemos de ajustar o tamanho do input para funcionar corretamente

# AlexNet

In [ ]:
AlexNet_model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(7, activation='softmax')
])

In [ ]:
AlexNet_model.summary()

In [ ]:
L = 'sparse_categorical_crossentropy'

AlexNet_model.compile(loss=L, optimizer="adam", metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=5, restore_best_weights=True)

history = AlexNet_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100,
  callbacks=[early_stopping_cb]
)

In [ ]:
AlexNet_model.evaluate(test_ds)

# MobileNet

In [ ]:
MobileNet_model = tf.keras.applications.mobilenet.MobileNet(input_shape = (224,224,3), weights = None ,classes = 7)

print(MobileNet_model)

In [ ]:
MobileNet_model.summary()

In [ ]:
L = 'sparse_categorical_crossentropy'

MobileNet_model.compile(loss=L, optimizer="adam", metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=5, restore_best_weights=True)

history = MobileNet_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100,
  callbacks=[early_stopping_cb]
)

In [ ]:
MobileNet_model.evaluate(test_ds)

# VGG-16

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16(input_shape = (224,224,3), weights = None ,classes = 7)

print(vgg16_model)

In [ ]:
vgg16_model.summary()

In [ ]:
L = 'sparse_categorical_crossentropy'

vgg16_model.compile(loss=L, optimizer="adam", metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=5, restore_best_weights=True)

history = vgg16_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100,
  callbacks=[early_stopping_cb]
)

In [ ]:
vgg16_model.evaluate(test_ds)